In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/marketing-campaigns-logistic-regression/Marketingcampaigns.csv


In [3]:
marketing_campaign_data = pd.read_csv('/kaggle/input/marketing-campaigns-logistic-regression/Marketingcampaigns.csv')
marketing_campaign_data.head()

,Customer id,Age,Gender,Location,Email Opened,Email Clicked,Product page visit,Discount offered,Purchased
0,1,22,0,Perth,1,1,3,1,1
1,2,55,0,Auckland,1,0,0,0,0
2,3,15,1,Sydney,0,1,2,1,1
3,4,25,0,Brisbane,1,1,5,1,0
4,5,36,1,Brisbane,0,1,1,1,0


In [14]:
marketing_campaign_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer id         20 non-null     int64 
 1   Age                 20 non-null     int64 
 2   Gender              20 non-null     int64 
 3   Location            20 non-null     object
 4   Email Opened        20 non-null     int64 
 5   Email Clicked       20 non-null     int64 
 6   Product page visit  20 non-null     int64 
 7   Discount offered    20 non-null     int64 
 8   Purchased           20 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 1.5+ KB


In [5]:
location = pd.get_dummies(marketing_campaign_data.Location)

In [6]:
marketing_data_concat = pd.concat([marketing_campaign_data, location],axis=1)
marketing_data_concat.head()

,Customer id,Age,Gender,Location,Email Opened,Email Clicked,Product page visit,Discount offered,Purchased,Auckland,Brisbane,Perth,Sydney
0,1,22,0,Perth,1,1,3,1,1,False,False,True,False
1,2,55,0,Auckland,1,0,0,0,0,True,False,False,False
2,3,15,1,Sydney,0,1,2,1,1,False,False,False,True
3,4,25,0,Brisbane,1,1,5,1,0,False,True,False,False
4,5,36,1,Brisbane,0,1,1,1,0,False,True,False,False


In [7]:
marketing_data_concat.drop('Location', inplace=True, axis=1)
marketing_data_concat.head()

,Customer id,Age,Gender,Email Opened,Email Clicked,Product page visit,Discount offered,Purchased,Auckland,Brisbane,Perth,Sydney
0,1,22,0,1,1,3,1,1,False,False,True,False
1,2,55,0,1,0,0,0,0,True,False,False,False
2,3,15,1,0,1,2,1,1,False,False,False,True
3,4,25,0,1,1,5,1,0,False,True,False,False
4,5,36,1,0,1,1,1,0,False,True,False,False


In [11]:
X = marketing_data_concat.drop(columns=['Purchased'])
Y = marketing_data_concat[['Purchased']]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.3, random_state=100)

In [20]:
print(X_train.shape)
print(X_test.shape)

(14, 11)
(6, 11)


In [16]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='liblinear', random_state=0)
lr_model.fit(X_train,Y_train)

Y_pred = lr_model.predict(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
lr_model.predict_proba(X_test)[:,0]

array([0.13293046, 0.10911784, 0.20779615, 0.35146942, 0.07794799,
       0.34544078])

In [23]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(Y_test, Y_pred)))

Model accuracy score: 0.8333


In [26]:
lr_model_100 = LogisticRegression(C=100, solver='liblinear', random_state=0)
lr_model_100.fit(X_train,Y_train)

Y_pred = lr_model_100.predict(X_test)
print(lr_model_100.predict_proba(X_test)[:,0])
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(Y_test, Y_pred)))

[1.20325538e-01 8.92306715e-04 8.94017905e-05 1.05446861e-04
 3.63625013e-04 9.64266556e-04]
Model accuracy score: 0.8333


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
lr_model_reg = LogisticRegression(C=0.01, solver='liblinear', random_state=0)
lr_model_reg.fit(X_train,Y_train)

Y_pred = lr_model_reg.predict(X_test)
print(lr_model_reg.predict_proba(X_test)[:,0])
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(Y_test, Y_pred)))

[0.45472285 0.50969389 0.56956797 0.66261678 0.44470599 0.57370035]
Model accuracy score: 0.5000


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])


Confusion matrix

 [[0 1]
 [0 5]]

True Positives(TP) =  0

True Negatives(TN) =  5

False Positives(FP) =  1

False Negatives(FN) =  0


In [28]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.83      1.00      0.91         5

    accuracy                           0.83         6
   macro avg       0.42      0.50      0.45         6
weighted avg       0.69      0.83      0.76         6



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]
classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)
print('Classification accuracy : {0:0.4f}'.format(classification_accuracy))

Classification accuracy : 0.8333


In [30]:
classification_error = (FP + FN) / float(TP + TN + FP + FN)
print('Classification error : {0:0.4f}'.format(classification_error))

Classification error : 0.1667


In [31]:
precision = TP / float(TP + FP)
print('Precision : {0:0.4f}'.format(precision))

Precision : 0.0000


In [32]:
recall = TP / float(TP + FN)
print('Recall or Sensitivity : {0:0.4f}'.format(recall))

Recall or Sensitivity : nan


/tmp/ipykernel_36/861198422.py:1: RuntimeWarning: invalid value encountered in divide
  recall = TP / float(TP + FN)


In [33]:
true_positive_rate = TP / float(TP + FN)
print('True Positive Rate : {0:0.4f}'.format(true_positive_rate))

True Positive Rate : nan


/tmp/ipykernel_36/289249085.py:1: RuntimeWarning: invalid value encountered in divide
  true_positive_rate = TP / float(TP + FN)


In [34]:
false_positive_rate = FP / float(FP + TN)
print('False Positive Rate : {0:0.4f}'.format(false_positive_rate))

False Positive Rate : 0.1667


In [35]:
specificity = TN / (TN + FP)
print('Specificity : {0:0.4f}'.format(specificity))

Specificity : 0.8333
